In [1]:
import os
import time
os.chdir("../")
from dataclasses import dataclass
from pathlib import Path
from zipfile import ZipFile
from DeepClassifier.constants import *
from DeepClassifier.utils import read_yaml, create_directories
from DeepClassifier.entities import DataIngestionConfig
import tensorflow as tf

In [2]:
@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data_dir: Path
    params_epochs: int
    params_batch_size: int
    params_augmentation: bool
    params_image_size: list
    params_validation_split: float
    params_rotation_range: float
    params_horizontal_flip: bool
    params_width_shift_range: float
    params_height_shift_range: float
    params_shear_range: float
    params_zoom_range: float

## Configuration Manager

In [3]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH
    ) -> None:
        """Inits ConfigurationManager.

        Args:
            config_file_path (Path, optional): Path of the config.yaml file.
                Defaults to the constant CONFIG_FILE_PATH.
            params_file_path (Path, optional): Path of the params.yaml file.
                Defaults to the constant PARAMS_FILE_PATH.
        """
        # Getting information in the config.yaml and params.yaml file
        self.config = read_yaml(yaml_file_path=config_file_path)
        self.params = read_yaml(yaml_file_path=params_file_path)
        
        # Creating the 'artifacts' directory
        create_directories(paths_of_directories=[self.config.artifacts_root])
    
    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        """Creates and returns PrepareCallbacksConfig.

        Returns:
            PrepareCallbacksConfig: The PrepareCallbacksConfig.
        """
        # Getting the values in the `prepare_callbacks` key of the config.yaml
        # file
        config = self.config.prepare_callbacks
        
        # Creating the directories 'artifacts/prepare_callbacks',
        # 'artifacts/prepare_callbacks/tensorboard_logs', and
        # 'artifacts/prepare_callbacks/checkpoint'
        checkpoint_model_dir = os.path.dirname(
            Path(config.checkpoint_model_filepath)
        )
        create_directories(
            paths_of_directories=[
                Path(config.root_dir),
                Path(config.tensorboard_root_log_dir),
                Path(checkpoint_model_dir)
            ]
        )
        
        # Creating and returning `PrepareCallbacksConfig`
        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return prepare_callbacks_config

    def get_training_config(self) -> TrainingConfig:
        """Creates and returns TrainingConfig.

        Returns:
            TrainingConfig: The TrainingConfig.
        """
        # Getting the values in the `training` key of the config.yaml
        # file
        config = self.config.training
        
        # Creating the directory 'artifacts/training'
        create_directories(paths_of_directories=[Path(config.root_dir)])
        
        # Getting the directory of the training data from the 'data ingestion'
        # key of the config.yaml file
        training_data_dir = os.path.join(
            self.config.data_ingestion.unzipped_file_dir,
            "PetImages",
        )
        
        # Creating and returning `TrainingConfig`
        training_config = TrainingConfig(
            root_dir=Path(config.root_dir),
            trained_model_path=Path(config.trained_model_path),
            updated_base_model_path=Path(
                self.config.prepare_base_model.updated_base_model_path
            ),
            training_data_dir=Path(training_data_dir),
            params_epochs=self.params.EPOCHS,
            params_batch_size=self.params.BATCH_SIZE,
            params_augmentation=self.params.AUGMENTATION,
            params_image_size=self.params.IMAGE_SIZE,
            params_validation_split=self.params.VALIDATION_SPLIT,
            params_rotation_range=self.params.ROTATION_RANGE,
            params_horizontal_flip=self.params.HORIZONTAL_FLIP,
            params_width_shift_range=self.params.WIDTH_SHIFT_RANGE,
            params_height_shift_range=self.params.HEIGHT_SHIFT_RANGE,
            params_shear_range=self.params.SHEAR_RANGE,
            params_zoom_range=self.params.ZOOM_RANGE,
        )
        return training_config

## Components

In [4]:
class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig) -> None:
        """Inits PrepareCallbacks.

        Args:
            config (PrepareCallbacksConfig): The PrepareCallbacksConfig.
        """
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d_%H-%M-%S")
        tb_running_log_dir = Path(os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_time_{timestamp}",
        ))
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_checkpoint_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True,
        )
        
    def get_tb_and_checkpoint_callbacks(self) -> list:
        return [
            self._create_tb_callbacks,
            self._create_checkpoint_callbacks
        ]

In [5]:
class Training:
    def __init__(self, config: TrainingConfig) -> None:
        """Inits Training.

        Args:
            config (TrainingConfig): The TrainingConfig.
        """
        self.config = config
    
    def get_updated_base_model(self):
        self.updated_base_model = tf.keras.models.load_model(
            filepath=self.config.updated_base_model_path
        )
    
    def train_val_generator(self):
        datagen_kwargs = dict(
            rescale=1./255,
            validation_split=self.config.params_validation_split,
        )
        
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
        )
        
        val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagen_kwargs
        )
        
        self.validation_generator = val_datagen.flow_from_directory(
            directory=self.config.training_data_dir,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs,
        )
        
        if self.config.params_augmentation:
            train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=self.config.params_rotation_range,
                horizontal_flip=self.config.params_horizontal_flip,
                width_shift_range=self.config.params_width_shift_range,
                height_shift_range=self.config.params_height_shift_range,
                shear_range=self.config.params_shear_range,
                zoom_range=self.config.params_zoom_range,
                **datagen_kwargs,
            )
        else:
            train_datagen = val_datagen
            
        self.train_generator = train_datagen.flow_from_directory(
            directory=self.config.training_data_dir,
            subset="training",
            shuffle=True,
            **dataflow_kwargs,
        )
    
    def train_model(self, callbacks: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.validation_generator.samples // self.validation_generator.batch_size
        
        self.trained_model = self.updated_base_model.fit(
            x=self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.validation_generator,
            callbacks=callbacks,
        )
        
        self.save_model(
            model=self.trained_model,
            path=self.config.trained_model_path,
        )

    @staticmethod
    def save_model(model: tf.keras.Model, path: Path):
        model.save(path)

In [6]:
try:
    config = ConfigurationManager()
    
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    callbacks = prepare_callbacks.get_tb_and_checkpoint_callbacks()

    training_config = config.get_training_config()

    training = Training(config=training_config)
    training.get_updated_base_model()
    training.train_val_generator()
    training.train_model(callbacks=callbacks)
except Exception as e:
    raise e

Found 4998 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
 164/1250 [==>...........................] - ETA: 1:54:39 - loss: 13.6802 - accuracy: 0.5404

c:\iNeuron\DeepCNNClassifier\env\lib\site-packages\PIL\TiffImagePlugin.py:864: UserWarning: Truncated File Read
  warnings.warn(str(msg))


 223/1250 [====>.........................] - ETA: 1:45:44 - loss: 12.6151 - accuracy: 0.5530

KeyboardInterrupt: 